In [1]:
import os

import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver
from pyrte_rrtmgp.all_skys_funcs import compute_profiles, create_gas_dataset, compute_clouds, compute_cloud_optics, expand_to_2d, combine_optical_props


rte_rrtmgp_dir = download_rrtmgp_data()

ncol = 24
nlay = 72
# Compute profiles
p_lay, t_lay, p_lev, t_lev, q, o3 = compute_profiles(300, ncol, nlay)

lw_no_aerosol = f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

p_lay_ref = ref_data["p_lay"].values.T
t_lay_ref = ref_data["t_lay"].values.T
p_lev_ref = ref_data["p_lev"].values.T
t_lev_ref = ref_data["t_lev"].values.T
q_ref = ref_data["h2o"].values.T
o3_ref = ref_data["o3"].values.T

p_lay_diff = np.abs(p_lay - p_lay_ref).max()
t_lay_diff = np.abs(t_lay - t_lay_ref).max()
p_lev_diff = np.abs(p_lev - p_lev_ref).max()
t_lev_diff = np.abs(t_lev - t_lev_ref).max()
q_diff = np.abs(q - q_ref).max()
o3_diff = np.abs(o3 - o3_ref).max()

p_lay_diff_percent = (np.abs(p_lay - p_lay_ref) / np.abs(p_lay_ref)).max() * 100
t_lay_diff_percent = (np.abs(t_lay - t_lay_ref) / np.abs(t_lay_ref)).max() * 100
p_lev_diff_percent = (np.abs(p_lev - p_lev_ref) / np.abs(p_lev_ref)).max() * 100
t_lev_diff_percent = (np.abs(t_lev - t_lev_ref) / np.abs(t_lev_ref)).max() * 100
q_diff_percent = (np.abs(q - q_ref) / np.abs(q_ref)).max() * 100
o3_diff_percent = (np.abs(o3 - o3_ref) / np.abs(o3_ref)).max() * 100

# Create xarray dataset with profiles
profiles = xr.Dataset(
    data_vars={
        "pres_layer": (["site", "layer"], p_lay),
        "temp_layer": (["site", "layer"], t_lay), 
        "pres_level": (["site", "level"], p_lev),
        "temp_level": (["site", "level"], t_lev),
        "water_vapor": (["site", "layer"], q),
        "ozone": (["site", "layer"], o3)
    }
)


gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
lw_clouds = os.path.join(rte_rrtmgp_dir, "rrtmgp-clouds-lw.nc")

# for sw
# sfc_alb_dir = 0.06
# sfc_alb_dif = 0.06
# mu0 = 0.86


# Create dataset with gas concentrations
gas_values = {
    'carbon_dioxide_GM': 348e-6,  # scalar
    'methane_GM': 1650e-9,  # scalar
    'nitrous_oxide_GM': 306e-9,  # scalar
    'nitrogen_GM': 0.7808,  # scalar
    'oxygen_GM': 0.2095,  # scalar
    'carbon_monoxide_GM': 0.0  # scalar
}

gases = create_gas_dataset(gas_values, ncol, nlay)

atmosphere = xr.merge([profiles, gases])
top_at_1 = atmosphere["pres_layer"].values[0,0]<atmosphere["pres_layer"].values[0,-1]

t_sfc = t_lev[:, nlay if top_at_1 else 0]
atmosphere["surface_temperature"] = xr.DataArray(t_sfc, dims=["site"])

gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
clear_sky_optical_props = gas_optics_lw.gas_optics.compute(atmosphere, problem_type="absorption", add_to_input=False)

cloud_optics = xr.load_dataset(lw_clouds)

t_sfc = t_lev[0, nlay if top_at_1 else 0]
emis_sfc = expand_to_2d(0.98, ncol, nlay+1, name="surface_emissivity")


lwp, iwp, rel, rei = compute_clouds(cloud_optics, ncol, nlay, p_lay, t_lay)

tau = compute_cloud_optics(lwp, iwp, rel, rei, cloud_optics)

# Create dataset with optical properties
clouds_optical_props = xr.Dataset(
    {
        "tau": (["site", "layer", "gpt"], tau),
    }
)

clear_sky_optical_props["surface_emissivity"] = 0.98


# Increment the optical properties
combine_optical_props(clouds_optical_props, clear_sky_optical_props)

solver = RTESolver()
fluxes = solver.solve(clear_sky_optical_props, add_to_input=False)


lw_no_aerosol = f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

# Print max differences and percentage differences
up_diff = np.abs(fluxes["lw_flux_up"].values - ref_data["lw_flux_up"].values.T)
down_diff = np.abs(fluxes["lw_flux_down"].values - ref_data["lw_flux_dn"].values.T)

print(f"Max difference in upward flux: {up_diff.max():.3f} W/m2")
print(f"Max difference in downward flux: {down_diff.max():.3f} W/m2")

up_pct = 100 * up_diff / ref_data["lw_flux_up"].values.T
down_pct = 100 * down_diff / ref_data["lw_flux_dn"].values.T

print(f"Max percentage difference in upward flux: {up_pct.max():.3f}%")
print(f"Max percentage difference in downward flux: {down_pct.max():.3f}%")

# Assertions
assert np.isclose(fluxes["lw_flux_up"].values, ref_data["lw_flux_up"].values.T, atol=2).all()
assert np.isclose(fluxes["lw_flux_down"].values, ref_data["lw_flux_dn"].values.T, atol=3).all()



Max difference in upward flux: 1.377 W/m2
Max difference in downward flux: 2.973 W/m2
Max percentage difference in upward flux: 1.199%
Max percentage difference in downward flux: nan%


/tmp/ipykernel_359553/3744780700.py:123: RuntimeWarning: invalid value encountered in divide
  down_pct = 100 * down_diff / ref_data["lw_flux_dn"].values.T


AssertionError: 